In [1]:
import numpy as np
import pandas as pd
import sys
import astropy.constants as ac
from tqdm import tqdm

In [2]:
K_B = ac.k_B.cgs.value
H = ac.h.cgs.value
C = ac.c.cgs.value


def percent_counter(z, nz, y=0, ny=1, x=0, nx=1):
    """ displays percentage completed of a long operation (usually a for loop) for up to three indices """

    percentage = float((x + nx * y + nx * ny * z) / (nx * ny * nz) * 100.0)
    #sys.stdout.write("calculating: {:.1f}%\r".format(percentage))
    sys.stdout.flush()


def calc_analyt_planck_in_interval(temp, lower_lambda, higher_lambda):
    """ calculates the planck function over a wavelength integral

    :param temp: float
                 the blackbody temperature

    :param lower_lambda: float
                         lower wavelength boundary (cm units!)

    :param higher_lambda: float
                          upper wavelength boundary (cm units!)

    :return: float
             the Planckian blackbody function integrated over a wavelength interval and averaged
    """

    d = 2.0 * (K_B / H)**3 * K_B * temp**4 / C**2
    y_top = H * C / (higher_lambda * K_B * temp)
    y_bot = H * C / (lower_lambda * K_B * temp)

    result = 0

    for n in range(1, 200):  # 200 found to be accurate enough.
        result += np.exp(-n*y_top) * (y_top**3/n + 3.0*y_top**2/n**2 + 6.0*y_top/n**3 + 6.0/n**4) \
                - np.exp(-n*y_bot) * (y_bot**3/n + 3.0*y_bot**2/n**2 + 6.0*y_bot/n**3 + 6.0/n**4)

    result *= d / (higher_lambda - lower_lambda)

    return result


def gauss_pdf(x, x_0, hwhm):

    pdf = (np.log(2)/np.pi)**0.5 / hwhm * np.exp(-np.log(2) * ((x - x_0) / hwhm)**2)

    return pdf


def convolve_with_gaussian(old_lamda, old_flux, resolution, new_lamda=None):

    if new_lamda is None:

        new_lamda = [old_lamda[0]]

        while new_lamda[-1] < old_lamda[-1]:

            new_lamda.append(new_lamda[-1] + new_lamda[-1] / resolution)

    # need wavelength widths of old lambda grid
    delta_lamda = np.zeros(len(old_lamda))

    for ll in range(len(old_lamda)):

        if ll == 0:

            delta_lamda[ll] = (old_lamda[ll + 1] - old_lamda[ll])

        elif ll == len(old_lamda) - 1:

            delta_lamda[ll] = (old_lamda[ll] - old_lamda[ll - 1])

        else:
            delta_lamda[ll] = (old_lamda[ll + 1] - old_lamda[ll - 1]) / 2

    # calculation of new flux values due to convolution with older ones
    flux_conv = np.zeros(len(new_lamda))

    for l in range(len(new_lamda)):

        percent_counter(l, len(new_lamda))

        # FWHM of Gaussian pdf equals the resolving power R (and thus HWHM = R/2)
        hwhm = new_lamda[l] / (2 * resolution)

        for ll in range(len(old_lamda)):

            if old_lamda[ll] - new_lamda[l] < -5 * hwhm:
                continue

            elif old_lamda[ll] - new_lamda[l] > 5 * hwhm:
                break

            else:
                flux_conv[l] += old_flux[ll] * gauss_pdf(new_lamda[l], old_lamda[ll], hwhm) * delta_lamda[ll]

    return new_lamda, flux_conv


def convert_spectrum(old_lambda, old_flux, new_lambda, int_lambda=None, type='linear', extrapolate_with_BB_T=0):
    """ converts a spectrum from one to another resolution. This method conserves energy. It is the real deal.

        :param old_lambda: list of float or numpy array
                           wavelength values of the old grid to be discarded. must be in ascending order!

        :param old_flux: list of float or numpy array
                         flux values of the old grid to be discarded.

        :param new_lambda: list of float or numpy array
                           wavelength values of the new output grid. must be in ascending order!

        :param int_lambda: (optional) list of float or numpy array
                           wavelength values of the interfaces of the new grid bins. must be in ascending order!
                           if not provided they are calculated by taking the middle points between the new_lambda values.

        :param type: (optional) 'linear' or 'log'
                     either linear interpolation or logarithmtic interpolation possible

        :param extrapolate_with_BB_T: (optional) float
                                      the out-of-boundary flux values will be extrapolated with a blackbody spectrum.
                                      set here the temperature. if not provided the out-of-boundary flux values are set to zero.

        :return: list of floats
                 flux values at the new wavelength grid points

        """

    if int_lambda is None:

        int_lambda = []

        int_lambda.append(new_lambda[0] - (new_lambda[1] - new_lambda[0]) / 2)

        for x in range(len(new_lambda) - 1):
            int_lambda.append((new_lambda[x + 1] + new_lambda[x]) / 2)

        int_lambda.append(new_lambda[-1] + (new_lambda[-1] - new_lambda[-2]) / 2)

    if extrapolate_with_BB_T > 0:

        extrapol_values = []

        #print("\n\nPre-tabulating blackbody values with a temperature of " + str(extrapolate_with_BB_T) + " K ...\n")

        for i in range(len(new_lambda)):

            percent_counter(i, len(new_lambda))

            extrapol_values.append(np.pi * calc_analyt_planck_in_interval(extrapolate_with_BB_T, int_lambda[i], int_lambda[i+1]))

        #print("Pre-tabulation done! \n")

    elif extrapolate_with_BB_T == 0:

        extrapol_values = np.zeros(len(new_lambda))

    else:
        raise ValueError("Error: extrapolation blackbody temperature cannot be negative.")

    #print("Starting pre-conversion...\n")

    int_flux = [0] * len(int_lambda)
    new_flux = []
    old_lambda = np.array(old_lambda)  # conversion required by np.where

    if type == 'linear':

        for i in range(len(int_lambda)):

            percent_counter(i, len(int_lambda))

            if int_lambda[i] < old_lambda[0]:
                continue

            elif int_lambda[i] > old_lambda[len(old_lambda) - 1]:
                break

            else:
                p_bot = len(np.where(old_lambda < int_lambda[i])[0]) - 1

                interpol = old_flux[p_bot] * (old_lambda[p_bot + 1] - int_lambda[i]) + old_flux[p_bot + 1] * (int_lambda[i] - old_lambda[p_bot])
                interpol /= (old_lambda[p_bot + 1] - old_lambda[p_bot])
                int_flux[i] = interpol

        #print("\n  Pre-conversion done!")

        #print("\nStarting main conversion...\n")

        for i in range(len(new_lambda)):

            percent_counter(i, len(new_lambda))

            if int_flux[i] == 0 or int_flux[i+1] == 0:
                new_flux.append(extrapol_values[i])

            else:
                p_bot = len(np.where(old_lambda < int_lambda[i])[0]) - 1

                p_start = p_bot + 1

                for p in range(p_start, len(old_lambda)):

                    if p == p_start:
                        if old_lambda[p_start] < int_lambda[i + 1]:
                            interpol = (int_flux[i] + old_flux[p]) / 2.0 * (old_lambda[p] - int_lambda[i])

                        else:
                            interpol = (int_flux[i] + int_flux[i + 1]) / 2.0
                            break
                    else:
                        if old_lambda[p] < int_lambda[i + 1]:
                            interpol += (old_flux[p - 1] + old_flux[p]) / 2.0 * (old_lambda[p] - old_lambda[p - 1])

                        else:
                            interpol += (old_flux[p - 1] + int_flux[i + 1]) / 2.0 * (int_lambda[i + 1] - old_lambda[p - 1])
                            interpol /= (int_lambda[i + 1] - int_lambda[i])
                            break

                new_flux.append(interpol)

    elif type == 'log':

        for i in range(len(int_lambda)):

            percent_counter(i, len(int_lambda))

            if int_lambda[i] < old_lambda[0]:
                continue

            elif int_lambda[i] > old_lambda[len(old_lambda) - 1]:
                break

            else:
                p_bot = len(np.where(old_lambda < int_lambda[i])[0]) - 1

                interpol = old_flux[p_bot] ** (old_lambda[p_bot + 1] - int_lambda[i]) * old_flux[p_bot + 1] ** (int_lambda[i] - old_lambda[p_bot])
                interpol = interpol**(1/(old_lambda[p_bot + 1] - old_lambda[p_bot]))
                int_flux[i] = interpol

        #print("\n  Pre-conversion done!")

        #print("\nStarting main conversion...\n")

        for i in range(len(new_lambda)):

            percent_counter(i, len(new_lambda))

            if int_flux[i] == 0 or int_flux[i + 1] == 0:
                new_flux.append(extrapol_values[i])

            else:
                p_bot = len(np.where(old_lambda < int_lambda[i])[0]) - 1

                p_start = p_bot + 1

                for p in range(p_start, len(old_lambda)):

                    if p == p_start:
                        if old_lambda[p_start] < int_lambda[i + 1]:
                            interpol = (int_flux[i] * old_flux[p]) ** (0.5 * (old_lambda[p] - int_lambda[i]))

                        else:
                            interpol = (int_flux[i] * int_flux[i + 1]) ** 0.5
                            break
                    else:
                        if old_lambda[p] < int_lambda[i + 1]:
                            interpol *= (old_flux[p - 1] * old_flux[p]) ** (0.5 * (old_lambda[p] - old_lambda[p - 1]))

                        else:
                            interpol *= (old_flux[p - 1] * int_flux[i + 1]) ** (0.5 * (int_lambda[i + 1] - old_lambda[p - 1]))
                            interpol = interpol ** (1/(int_lambda[i + 1] - int_lambda[i]))
                            break

                new_flux.append(interpol)

    #print("\n  Main conversion done!")

    return new_flux


def rebin_spectrum_to_resolution(old_lamda, old_flux, resolution, w_unit='cm', type='linear'):
    """ rebins a given spectrum to a new resolution

    :param old_lambda:  list of float or numpy array
                           wavelength values of the old grid to be discarded.
                           must be in ascending order!

    :param old_flux:    list of float or numpy array
                        flux values of the old grid to be discarded.

    :param resolution:  float
                        resolution (R=lamda/delta_lamda) of the new, rebinned wavelength grid

    :param w_unit:      'cm'/'micron'
                        the units of the old wavelength values. will be the output units as well.

    :param type:        (optional) 'linear', 'log' or 'gaussian'
                        - linear interpolation is the default. it conserves the total energy in each bin.
                        - logarithmic interpolation is usually used for opacities or other quantities where the integral does not need to be conserved.
                        - alternatively, the spectrum can be convolved with a Gaussian distribution, where FWHM = R

    :return 1:          wavelength values of the rebinned grid
    :return 2:          flux values of the rebinned grid
    """

    if w_unit == 'micron':
        old_lamda = [l * 1e-4 for l in old_lamda]

    bot_limit = old_lamda[0]

    top_limit = old_lamda[-1]

    rebin_lamda = []
    l_point = bot_limit

    while l_point < top_limit:
        rebin_lamda.append(l_point)

        l_point *= (resolution + 1) / resolution

    if type == "gaussian":
        _, rebin_flux = convolve_with_gaussian(old_lamda, old_flux, resolution, rebin_lamda)
    else:
        rebin_flux = convert_spectrum(old_lamda, old_flux, rebin_lamda, type=type, extrapolate_with_BB_T=0)

    if w_unit == 'micron':
        rebin_lamda = [l * 1e4 for l in rebin_lamda]

    return rebin_lamda, rebin_flux



In [3]:
"""
# There are 28 pressure points and then there's one line that corresponds to the wavelength
df_test = pd.read_csv('/media/imalsky/Samsung_T5/Opacities-Hayley/opacCO.dat',
                 delim_whitespace= True, dtype=np.float64,
                 names=['P','500', '600', '700', '800', '900',
                        '1000','1100', '1200', '1300', '1400', '1500', '1600','1700', '1800', '1900',
                        '2000', '2100', '2200', '2300', '2400','2500', '2600', '2700', '2800', '2900',
                        '3000', '3100', '3200', '3300', '3400', '3500', '3600', '3700', '3800', '3900',
                        '4000', '4100', '4200', '4300', '4400', '4500', '4600', '4700', '4800', '4900',
                        '5000'], skiprows=2)
"""

"\n# There are 28 pressure points and then there's one line that corresponds to the wavelength\ndf_test = pd.read_csv('/media/imalsky/Samsung_T5/Opacities-Hayley/opacCO.dat',\n                 delim_whitespace= True, dtype=np.float64,\n                 names=['P','500', '600', '700', '800', '900',\n                        '1000','1100', '1200', '1300', '1400', '1500', '1600','1700', '1800', '1900',\n                        '2000', '2100', '2200', '2300', '2400','2500', '2600', '2700', '2800', '2900',\n                        '3000', '3100', '3200', '3300', '3400', '3500', '3600', '3700', '3800', '3900',\n                        '4000', '4100', '4200', '4300', '4400', '4500', '4600', '4700', '4800', '4900',\n                        '5000'], skiprows=2)\n"

In [4]:
#n_pressure_points = 28
#n_temperature_points = 46

#print (len(df_test.index) / (n_pressure_points + 1))

In [10]:
# This is the part where I read in the 3.5 GB file

n_wavelengths = 176842 # this is the total number
n_pressure_points = 28
n_temperature_points = 46


# There are 28 pressure points and then there's one line that corresponds to the wavelength
df = pd.read_csv('/media/imalsky/Samsung_T5/Opacities-Hayley/opacCO.dat',
                 delim_whitespace= True, dtype=np.float64,
                 names=['P','500', '600', '700', '800', '900',
                        '1000','1100', '1200', '1300', '1400', '1500', '1600','1700', '1800', '1900',
                        '2000', '2100', '2200', '2300', '2400','2500', '2600', '2700', '2800', '2900',
                        '3000', '3100', '3200', '3300', '3400', '3500', '3600', '3700', '3800', '3900',
                        '4000', '4100', '4200', '4300', '4400', '4500', '4600', '4700', '4800', '4900',
                        '5000'], skiprows=2, nrows=n_wavelengths * (n_pressure_points + 1))

pd.set_option('display.max_rows', df.shape[0])
pd.set_option("display.precision", 8)


wavelengths = list(df[df.index % (n_pressure_points + 1) == 0].P)
data = df[df.index % (n_pressure_points + 1) != 0]


pressures    = list(data.P[0:n_pressure_points])
temperatures = ['500', '600', '700', '800', '900',
            '1000','1100', '1200', '1300', '1400', '1500', '1600','1700', '1800', '1900',
            '2000', '2100', '2200', '2300', '2400','2500', '2600', '2700', '2800', '2900',
            '3000', '3100', '3200', '3300', '3400', '3500', '3600', '3700', '3800', '3900',
            '4000', '4100', '4200', '4300', '4400', '4500', '4600', '4700', '4800', '4900',
            '5000']

data = data.drop(['P'], axis=1)
data = data.reset_index(drop=True)

np_data = data.to_numpy()

data_dictionary = {}

# Create a dictionary where each key corresponds to the opacities for a pressure-temperature pair
# They corresponding lists are all the wavelengths
# So the dictionary key for a pressure of 10 and a temperature of 500 will be a list, and the order 
# of that list corresponds to all the wavelengths needed
for w in range(n_wavelengths):
    for p in range(len(pressures)):
        for t in range(len(temperatures)):
            key_str = str(pressures[p]) + '_' + str(temperatures[t])
            data_dictionary.setdefault(key_str, []).append(np_data[p + (w * 28)][t])

In [ ]:
data_new = np.zeros((n_wavelengths * (n_pressure_points), n_temperature_points + 1))

w = 0
# now make an np array with all the new opacity values
for p in tqdm(range(len(pressures))):
    pressure = pressures[p]

    for t in range(len(temperatures)):
        
        print (t)
        temperature = temperatures[t]
        key_string = str(pressure)+ '_' + str(temperature)

        old_opac  = data_dictionary[key_string]
        old_lambda = wavelengths

        new_lambda, new_opac = rebin_spectrum_to_resolution(old_lambda, old_opac, 125000, w_unit='cm', type='linear')

        if new_opac[0] == 0.0:
            new_opac[0] = new_opac[1]
        else:
            pass
        
        for w in range(len(new_opac)):
            data_new[p + w*28][t + 1] = new_opac[w]




  0%|          | 0/28 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [ ]:
for w in range(len(new_opac)):
    
    for p in range(n_pressure_points):
        data_new[p + w*28][0] = pressures[p]

data_new = data_new[~np.all(data_new == 0, axis=1)]

np.savetxt('/home/imalsky/Desktop/new-CO.txt', data_new, fmt='%1.6E')

import fileinput
i = 0
for linenum,line in enumerate( fileinput.FileInput('/home/imalsky/Desktop/test.txt',inplace=1) ):
    if linenum == 0:
        print ("500.000 600.000 700.000 800.000 900.000 1000.000 1100.000 1200.000 1300.000 1400.000 1500.000 1600.000 1700.000 1800.000 1900.000 2000.000 2100.000 2200.000 2300.000 2400.000 2500.000 2600.000 2700.000 2800.000 2900.000 3000.000 3100.000 3200.000 3300.000 3400.000 3500.000 3600.000 3700.000 3800.000 3900.000 4000.000 4100.000 4200.000 4300.000 4400.000 4500.000 4600.000 4700.000 4800.000 4900.000 5000.000 ")
        print ("1.000000E-01 2.154435E-01 4.641589E-01 1.000000E+00 2.154435E+00 4.641589E+00 1.000000E+01 2.154435E+01 4.641589E+01 1.000000E+02 2.154435E+02 4.641589E+02 1.000000E+03 2.154435E+03 4.641589E+03 1.000000E+04 2.154435E+04 4.641589E+04 1.000000E+05 2.154435E+05 4.641589E+05 1.000000E+06 2.154435E+06 4.641589E+06 1.000000E+07 2.154435E+07 4.641589E+07 1.000000E+08")
        #print (line.rstrip())
        
    if linenum%28 == 0:
        print ("{0:.9E}".format(new_lambda[i]))
        print (line.rstrip())
        i = i + 1
    else:
        print (line.rstrip())
    